In [2]:
import numpy as np
import importlib
import classA_U1FGTN
importlib.reload(classA_U1FGTN)
from classA_U1FGTN import classA_U1FGTN
from matplotlib import pyplot as plt
import os
import time

In [ ]:
# --- Cell 2: basic config & helper for clean prints ---
Nx, Ny    = 8, 8        # keep small for quick tests; bump later (e.g., 21)
cycles    = 10              # short run for smoke test
samples   = 10             # small sample count to keep it snappy
n_jobs    = 10              # try <= samples
backend   = "threading"         # "loky" (processes) or "threading"
nshell    = None           # whatever you usually use
DW        = True           # enable domain walls (to exercise smart x0 picking)

def banner(msg):
    print("\n" + "="*len(msg))
    print(msg)
    print("="*len(msg))

In [7]:
# --- Cell 3: instantiate the model ---
m = classA_U1FGTN(Nx=Nx, Ny=Ny, DW=DW, cycles=cycles, samples=samples, nshell=nshell)

DWs at x=(2, 5)
------------------------- classA_U1FGTN Initialized -------------------------


In [8]:
# --- Cell 4: run entanglement contour DYNAMICS (gif + final frames) ---
outdir_dyn = None
start = time.time()
banner("Running entanglement_contour_dynamics()")
outdir_dyn = m.entanglement_contour_dynamics(
    samples=samples,
    n_jobs=n_jobs,
    backend=backend,        # "loky" strongly recommended for reproducibility
    save_data=True,
    filename_prefix=f"smoketest_N{Nx}_S{samples}_J{n_jobs}_{backend}"
)
elapsed = time.time() - start
print(f"\nSaved dynamics outputs in:\n  {outdir_dyn}\nElapsed: {elapsed:.2f}s")

# quick peek: list files produced
print("\nArtifacts in outdir:")
for f in sorted(os.listdir(outdir_dyn)):
    if f.startswith("smoketest"):
        print("  -", f)


Running entanglement_contour_dynamics()


ValueError: Invalid backend: threads, expected one of ['loky', 'multiprocessing', 'sequential', 'threading']

In [ ]:
# --- Cell 5: preview final frames from the dynamics NPZ (optional quick check) ---
# (This cell expects the default save naming from the previous call.)
npz_path = os.path.join(
    outdir_dyn,
    f"smoketest_N{Nx}_S{samples}_J{n_jobs}_{backend}_data.npz"
)
data = np.load(npz_path)
avg_maps  = data["avg_maps"]          # shape (T, Nx, Ny)
traj_maps = data["traj_maps"]         # shape (S, T, Nx, Ny)

banner("Preview final frames (traj[0] and avg)")
fig, axs = plt.subplots(1, 2, figsize=(8.5, 3.8), constrained_layout=True)
im0 = axs[0].imshow(traj_maps[0, -1], cmap="Blues", origin="upper", aspect="equal")
axs[0].set_title("Final s(r) — traj #0")
im1 = axs[1].imshow(avg_maps[-1], cmap="Blues", origin="upper", aspect="equal")
axs[1].set_title("Final s(r) — sample-avg")
for ax in axs:
    ax.set_xlabel("y"); ax.set_ylabel("x")
fig.colorbar(im0, ax=axs[0], fraction=0.046, pad=0.04)
fig.colorbar(im1, ax=axs[1], fraction=0.046, pad=0.04)
plt.show()

In [ ]:
# --- Cell 6: run entanglement contour Y-PROFILES (2x2 figure) ---
banner("Running entanglement_contour_yprofiles()")
out_pdf = m.entanglement_contour_yprofiles(
    samples=samples,
    n_jobs=n_jobs,
    backend=backend,
    save_data=True,
    filename=None,          # let it auto-name the PDF
)
print("\nSaved PDF:\n ", out_pdf)

# also show the saved data file path
data_dir = os.path.dirname(out_pdf)
prefix   = f"entanglement_yprofiles_data_N{Nx}_S{samples}_J{n_jobs}_{backend}.npz"
npz_file = os.path.join(data_dir, prefix)
print("Saved data:\n ", npz_file)

In [ ]:
# --- Cell 7: sanity-check y-profiles data and plot one panel inline ---
if os.path.exists(npz_file):
    d = np.load(npz_file)
    t_vals  = d["t_vals"]                 # (T,)
    xs      = d["x_positions"]            # (num_x,)
    trajM   = d["traj_profiles_mean"]     # (num_x, T)
    avgM    = d["avg_profiles"]           # (num_x, T)

    banner("Quick profile preview")
    fig, ax = plt.subplots(figsize=(6.5, 3.8))
    for i, x0 in enumerate(xs[:min(5, len(xs))]):   # show a few traces
        ax.plot(t_vals, trajM[i], label=f"x0={int(x0)}")
    ax.set_xlabel("cycle t")
    ax.set_ylabel(r"$\sum_y s(x_0,y)$")
    ax.set_title(r"Traj-resolved mean $\overline{s_{G_2}}$ (subset)")
    ax.grid(True, alpha=0.3)
    ax.legend(fontsize=8, ncols=2)
    plt.show()
else:
    print("Data npz not found (did save_data=True?).")

In [ ]:
# --- Cell 8: quick A/B test of backends (optional) ---
# WARNING: "threading" shares RNG & object state; use for speed only, not reproducibility.
for bk in ("threading", "loky"):
    banner(f"Backend check: {bk}")
    m.entanglement_contour_dynamics(
        samples=2, n_jobs=2, backend=bk,
        save_data=False, filename_prefix=f"bkchk_{bk}_N{Nx}"
    )
print("Backend check done.")